In [1]:
!pip install vncorenlp
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 34.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645932 sha256=cd7108b845bba72a2a37b511ca7b5a8c6eaecaa1dc0e54bafa11f40afdaebaa6
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp
--2023-12-26 15:05:56--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: 'VnCoreNLP-1.1.1.jar'

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  --.-KB/s    in 0.1s    

2023-12-26 15:05:58 (205 MB/s) - 

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import json
from vncorenlp import VnCoreNLP
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm

In [3]:
raw_dataset_law_train = pd.read_json("/kaggle/input/datasetlawvn/train.json")
raw_dataset_law_val = pd.read_json("/kaggle/input/datasetlawvn/val.json")

In [4]:
raw_dataset_law = pd.concat([raw_dataset_law_train, raw_dataset_law_val], ignore_index=True)

In [6]:
topics_dataset_with_subquestion_raw = raw_dataset_law[['subtopic', 'topic']]

In [7]:
topics_dataset_with_question_raw = raw_dataset_law[['question', 'topic']]

In [14]:
topics_dataset = topics_dataset_with_subquestion_raw

final_df = pd.DataFrame(columns=['Index', 'Subtopic', 'Topic'])

for i in tqdm(range(len(topics_dataset)), desc="Processing"):
    subtopic_data = topics_dataset['subtopic'].iloc[i]
    topic_subtopic = topics_dataset['topic'].iloc[i]
    
    subtopic_data = {key: value for key, value in subtopic_data.items() if value is not None}
    
    subtopic_df = pd.DataFrame(list(subtopic_data.items()), columns=['Index', 'Subtopic'])
    
    topic_df = pd.DataFrame({'Topic': [topic_subtopic] * len(subtopic_df)})
    
    result_df = pd.concat([subtopic_df, topic_df], axis=1)
    
    final_df = pd.concat([final_df, result_df], ignore_index=True)

print(final_df)

Processing: 100%|██████████| 36070/36070 [02:29<00:00, 240.51it/s]

       Index                                           Subtopic  \
0          0  Khi doanh nghiệp trong nước hợp tác với nhà đầ...   
1          1  Nhà đầu tư nước ngoài được áp dụng điều kiện t...   
2          2  Khi nhà đầu tư nước ngoài và nhà đầu tư trong ...   
3          0  Phẫu thuật làm mỏm cụt ngón được hiểu như thế ...   
4          1  Khi nào người bệnh được phẫu thuật làm mỏm cụt...   
...      ...                                                ...   
111585     1  Thực hiện cấp lại giấy phép sử dụng tần số và ...   
111586     2  Trình tự cấp lại giấy phép sử dụng tần số và t...   
111587     0  Việc trẻ em khuyết tật bị tách tời khỏi cha mẹ...   
111588     1  Cha mẹ bị hạn chế quyền đối với con là trẻ em ...   
111589     2  Những ai có quyền yêu cầu Tòa án hạn chế quyền...   

                                                    Topic  
0                                 [Nhà đầu tư nước ngoài]  
1                                 [Nhà đầu tư nước ngoài]  
2              

In [15]:
final_df.rename(columns={'Subtopic': 'question'}, inplace=True)
final_df.rename(columns={'Topic': 'topic'}, inplace=True)
final_df

,Index,question,topic
0,0,Khi doanh nghiệp trong nước hợp tác với nhà đầ...,[Nhà đầu tư nước ngoài]
1,1,Nhà đầu tư nước ngoài được áp dụng điều kiện t...,[Nhà đầu tư nước ngoài]
2,2,Khi nhà đầu tư nước ngoài và nhà đầu tư trong ...,[Nhà đầu tư nước ngoài]
3,0,Phẫu thuật làm mỏm cụt ngón được hiểu như thế ...,[Chấn thương chỉnh hình]
4,1,Khi nào người bệnh được phẫu thuật làm mỏm cụt...,[Chấn thương chỉnh hình]
...,...,...,...
111585,1,Thực hiện cấp lại giấy phép sử dụng tần số và ...,[Cấp Giấy phép sử dụng tần số và thiết bị vô t...
111586,2,Trình tự cấp lại giấy phép sử dụng tần số và t...,[Cấp Giấy phép sử dụng tần số và thiết bị vô t...
111587,0,Việc trẻ em khuyết tật bị tách tời khỏi cha mẹ...,[Trẻ em khuyết tật]
111588,1,Cha mẹ bị hạn chế quyền đối với con là trẻ em ...,[Trẻ em khuyết tật]


In [18]:
topics_dataset_raw = pd.concat([final_df, topics_dataset_with_question_raw], ignore_index=True)
topics_dataset_raw

,Index,question,topic
0,0,Khi doanh nghiệp trong nước hợp tác với nhà đầ...,[Nhà đầu tư nước ngoài]
1,1,Nhà đầu tư nước ngoài được áp dụng điều kiện t...,[Nhà đầu tư nước ngoài]
2,2,Khi nhà đầu tư nước ngoài và nhà đầu tư trong ...,[Nhà đầu tư nước ngoài]
3,0,Phẫu thuật làm mỏm cụt ngón được hiểu như thế ...,[Chấn thương chỉnh hình]
4,1,Khi nào người bệnh được phẫu thuật làm mỏm cụt...,[Chấn thương chỉnh hình]
...,...,...,...
147655,NaN,Kho bạc Nhà nước tham gia giao dịch công cụ nợ...,[Kho bạc Nhà nước]
147656,NaN,Tôi nhờ Ban tư vấn hướng dẫn giúp tôi về trình...,[Dự án đầu tư xây dựng]
147657,NaN,Cho tôi hỏi đối tượng phục vụ của Trung tâm tư...,[Trung tâm tư vấn pháp luật]
147658,NaN,Cho tôi hỏi thực hiện cấp lại giấy phép sử dụn...,[Cấp Giấy phép sử dụng tần số và thiết bị vô t...


In [19]:
topics_dataset_raw.shape

(147660, 3)

In [20]:
topics_dataset_raw.to_csv('topics_dataset_raw.csv', index=False, encoding='utf-8')

In [36]:
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [28]:
topics_dataset_clean = topics_dataset_raw.dropna().reset_index(drop=True)
print(topics_dataset_clean)

       Index                                           question  \
0          0  Khi doanh nghiệp trong nước hợp tác với nhà đầ...   
1          1  Nhà đầu tư nước ngoài được áp dụng điều kiện t...   
2          2  Khi nhà đầu tư nước ngoài và nhà đầu tư trong ...   
3          0  Phẫu thuật làm mỏm cụt ngón được hiểu như thế ...   
4          1  Khi nào người bệnh được phẫu thuật làm mỏm cụt...   
...      ...                                                ...   
111480     1  Thực hiện cấp lại giấy phép sử dụng tần số và ...   
111481     2  Trình tự cấp lại giấy phép sử dụng tần số và t...   
111482     0  Việc trẻ em khuyết tật bị tách tời khỏi cha mẹ...   
111483     1  Cha mẹ bị hạn chế quyền đối với con là trẻ em ...   
111484     2  Những ai có quyền yêu cầu Tòa án hạn chế quyền...   

                                                    topic  
0                                 [Nhà đầu tư nước ngoài]  
1                                 [Nhà đầu tư nước ngoài]  
2              

In [29]:
# unique topics
topics_raw = topics_dataset_clean['topic']
all_topics = []
for topic in topics_raw:
    if topic is not None:
        all_topics.extend(topic)
        
print(len(set(all_topics)))

8582


In [30]:
multilabel_topics = MultiLabelBinarizer()

In [31]:
topics_mlb = multilabel_topics.fit_transform(topics_raw)

In [32]:
topics_mlb.shape

(111485, 8582)

In [33]:
topics = pd.DataFrame(topics_mlb, columns=multilabel_topics.classes_)

In [34]:
topics

,Amoniac lỏng tổng hợp,An ninh hàng hải,An ninh hàng không,An ninh mạng,An ninh nguồn nước,An ninh quốc gia,An ninh thông tin,An ninh thông tin mạng,An ninh trật tự,An sinh xã hội,...,Ủy thác đầu tư,Ủy thác đầu tư gián tiếp ra nước ngoài,Ủy viên Bộ Chính trị,Ủy viên thường trực,Ứng dụng VNeID,Ứng dụng VssID,Ứng dụng công nghệ cao,Ứng dụng công nghệ thông tin,Ứng phó sự cố môi trường,Ứng xử nơi cư trú
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
segmenter_topics_dataset = topics_dataset_clean.copy()
segmenter_topics_dataset['question'] = segmenter_topics_dataset['question'].apply(lambda x: ' '.join([' '.join(y) for y in rdrsegmenter.tokenize(x)]))

In [38]:
datalaw_topics = pd.concat([segmenter_topics_dataset['question'], topics], axis=1)

In [39]:
topics_columns = datalaw_topics.columns

In [40]:
segmented_datalaw_topics = datalaw_topics.copy()

for column in segmented_datalaw_topics.columns:
    segmented_datalaw_topics.rename(columns={column: ' '.join([' '.join(y) for y in rdrsegmenter.tokenize(column)])}, inplace=True)

print(segmented_datalaw_topics.columns)

Index(['question', 'Amoniac lỏng tổng_hợp', 'An_ninh hàng_hải',
       'An_ninh hàng_không', 'An_ninh mạng', 'An_ninh nguồn nước',
       'An_ninh quốc_gia', 'An_ninh thông_tin', 'An_ninh thông_tin mạng',
       'An_ninh trật_tự',
       ...
       'Uỷ_thác đầu_tư', 'Uỷ_thác đầu_tư gián_tiếp ra nước_ngoài',
       'Uỷ_viên Bộ_Chính_trị', 'Uỷ_viên thường_trực', 'Ứng_dụng VNeID',
       'Ứng_dụng VssID', 'Ứng_dụng công_nghệ_cao',
       'Ứng_dụng công_nghệ_thông_tin', 'Ứng_phó sự_cố môi_trường',
       'Ứng_xử nơi cư_trú'],
      dtype='object', length=8583)


In [41]:
segmented_datalaw_topics

,question,Amoniac lỏng tổng_hợp,An_ninh hàng_hải,An_ninh hàng_không,An_ninh mạng,An_ninh nguồn nước,An_ninh quốc_gia,An_ninh thông_tin,An_ninh thông_tin mạng,An_ninh trật_tự,...,Uỷ_thác đầu_tư,Uỷ_thác đầu_tư gián_tiếp ra nước_ngoài,Uỷ_viên Bộ_Chính_trị,Uỷ_viên thường_trực,Ứng_dụng VNeID,Ứng_dụng VssID,Ứng_dụng công_nghệ_cao,Ứng_dụng công_nghệ_thông_tin,Ứng_phó sự_cố môi_trường,Ứng_xử nơi cư_trú
0,Khi doanh_nghiệp trong nước hợp_tác với nhà_đầ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Nhà_đầu_tư nước_ngoài được áp_dụng điều_kiện t...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Khi nhà_đầu_tư nước_ngoài và nhà_đầu_tư trong ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Phẫu_thuật làm mỏm cụt ngón được hiểu như_thế_...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Khi nào người_bệnh được phẫu_thuật làm mỏm cụt...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111480,Thực_hiện cấp lại giấy_phép sử_dụng tần_số và ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111481,Trình_tự cấp lại giấy_phép sử_dụng tần_số và t...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111482,Việc trẻ_em khuyết_tật bị tách tời khỏi cha_mẹ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111483,Cha_mẹ bị hạn_chế quyền đối_với con là trẻ_em ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
topics_columns = segmented_datalaw_topics.columns

In [43]:
duplicated_columns = segmented_datalaw_topics.columns[segmented_datalaw_topics.columns.duplicated()]

if len(duplicated_columns) > 0:
    print(f"Duplicated columns: {duplicated_columns}")
else:
    print("All columns are unique.")

Duplicated columns: Index(['Bến_thuỷ nội_địa', 'Cai_nghiện ma_tuý', 'Chăm_sóc sức_khoẻ',
       'Cục Đường_thuỷ nội_địa Việt_Nam', 'Hạ_tầng đường_thuỷ nội_địa',
       'Trung_tâm Văn_hoá Việt_Nam', 'Đường_thuỷ nội_địa'],
      dtype='object')


In [44]:
segmented_datalaw_topics = segmented_datalaw_topics.loc[:, ~segmented_datalaw_topics.columns.duplicated()]
print(segmented_datalaw_topics.columns)

Index(['question', 'Amoniac lỏng tổng_hợp', 'An_ninh hàng_hải',
       'An_ninh hàng_không', 'An_ninh mạng', 'An_ninh nguồn nước',
       'An_ninh quốc_gia', 'An_ninh thông_tin', 'An_ninh thông_tin mạng',
       'An_ninh trật_tự',
       ...
       'Uỷ_thác đầu_tư', 'Uỷ_thác đầu_tư gián_tiếp ra nước_ngoài',
       'Uỷ_viên Bộ_Chính_trị', 'Uỷ_viên thường_trực', 'Ứng_dụng VNeID',
       'Ứng_dụng VssID', 'Ứng_dụng công_nghệ_cao',
       'Ứng_dụng công_nghệ_thông_tin', 'Ứng_phó sự_cố môi_trường',
       'Ứng_xử nơi cư_trú'],
      dtype='object', length=8576)


In [45]:
topics_columns = segmented_datalaw_topics.columns

In [ ]:
import json
from tqdm import tqdm

data = segmented_datalaw_topics.to_dict(orient='records')

final_json = []

for item in tqdm(data, desc="Processing"):
    row = {}
    for column in topics_columns:
        row[column] = item.get(column, 0)
    final_json.append(row)

with open('topics_datalaw.json', 'w', encoding='utf-8') as json_file:
    json.dump(final_json, json_file, ensure_ascii=False, indent=3)